# Convolutional Neural Network

### Importing the libraries

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2025-05-12 19:18:32.287165: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
tf.__version__

'2.13.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [3]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

train_generator = train_datagen.flow_from_directory(
    'dataset/training_set',  # <-- Corrected path
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)


Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [4]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = train_datagen.flow_from_directory(
    'dataset/test_set',  # <-- Corrected path
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [5]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [6]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(64, 64, 3)))

### Step 2 - Pooling

In [7]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=2))

### Adding a second convolutional layer

In [8]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=2))

### Step 3 - Flattening

In [9]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [10]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [11]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [12]:
cnn.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [13]:
cnn.fit(x=train_generator, validation_data=test_generator, epochs=30)

Epoch 1/30
250/250 [==============================] - 44s 174ms/step - loss: 0.6621 - accuracy: 0.5974 - val_loss: 0.6120 - val_accuracy: 0.6840
Epoch 2/30
250/250 [==============================] - 37s 148ms/step - loss: 0.5904 - accuracy: 0.6885 - val_loss: 0.6092 - val_accuracy: 0.6875
Epoch 3/30
250/250 [==============================] - 42s 169ms/step - loss: 0.5570 - accuracy: 0.7153 - val_loss: 0.5696 - val_accuracy: 0.7015
Epoch 4/30
250/250 [==============================] - 46s 182ms/step - loss: 0.5314 - accuracy: 0.7285 - val_loss: 0.5447 - val_accuracy: 0.7250
Epoch 5/30
250/250 [==============================] - 44s 175ms/step - loss: 0.5137 - accuracy: 0.7380 - val_loss: 0.5205 - val_accuracy: 0.7520
Epoch 6/30
250/250 [==============================] - 40s 160ms/step - loss: 0.4946 - accuracy: 0.7599 - val_loss: 0.5018 - val_accuracy: 0.7670
Epoch 7/30
250/250 [==============================] - 42s 166ms/step - loss: 0.4714 - accuracy: 0.7765 - val_loss: 0.4896 - val_ac

## Part 4 - Making a single prediction

In [20]:
import numpy as np
from tensorflow.keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/dog.1.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
train_generator.class_indices
if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'

1/1 [==============================] - 0s 19ms/step


In [21]:
print(prediction)

dog


In [22]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Use test_datagen, not train_datagen
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    'dataset/test_set',         # path to your test dataset
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary',
    shuffle=False               # keep it False to match predictions with true labels
)


Found 2000 images belonging to 2 classes.


In [23]:
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Make predictions on the entire test set
predictions = cnn.predict(test_generator)

# Convert predictions to class labels (0 or 1)
predicted_classes = (predictions > 0.5).astype(int).reshape(-1)

# Get true labels
true_classes = test_generator.classes

# Get class label mapping
class_labels = list(test_generator.class_indices.keys())

# Evaluate accuracy
accuracy = accuracy_score(true_classes, predicted_classes)
print(f"✅ Accuracy on test set: {accuracy * 100:.2f}%")

# Optional: More detailed evaluation
print("\n📊 Classification Report:")
print(classification_report(true_classes, predicted_classes, target_names=class_labels))

print("🧩 Confusion Matrix:")
print(confusion_matrix(true_classes, predicted_classes))

63/63 [==============================] - 4s 64ms/step
✅ Accuracy on test set: 80.30%

📊 Classification Report:
              precision    recall  f1-score   support

        cats       0.82      0.78      0.80      1000
        dogs       0.79      0.83      0.81      1000

    accuracy                           0.80      2000
   macro avg       0.80      0.80      0.80      2000
weighted avg       0.80      0.80      0.80      2000

🧩 Confusion Matrix:
[[775 225]
 [169 831]]
